# Regression of Time of Day Accidents Occurred vs. Day of the Week Accidents Occurred

In [1]:
# Data Science Libraries
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Regression/Modelling libraries
from scipy.stats import linregress